In [10]:
# Install dependencies

%pip install -qU langchain langchain-openai langchain-community pypdf

Note: you may need to restart the kernel to use updated packages.


Set the environment variables. We need an API key and an LLM server, at minimum.

In [1]:
import os

# We will add the api key as an environment variable.
# You can set this value directrly form the shell,
# instead of explicitly setting it on code.
os.environ["OPENAI_API_KEY"] = "sk-h8JtQkCCJUOy-TAdDxCLGw"
# We need a custom endpoint, as we will be calling Verde's LLM
API_ENDPOINT = "https://llm-api.cyverse.ai"

The first step is to create a `Chat` object. This will allow us to call it through langchain and later on using `LCEL`

In [2]:
from langchain_openai import ChatOpenAI 		# We use the OpenAI protocol, but are using another provider (Verde)

# We will connect to Mistral Instruct v0.3 through Verde
# Notice how we need to specify the API endpoint
model = ChatOpenAI(model="Meta-Llama-3.1-70B-Instruct-quantized", base_url=API_ENDPOINT)

# Do a test call
from pprint import pprint
response = model.invoke("Hello, who are you?")

pprint(response)

AIMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 41, 'total_tokens': 64, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'neuralmagic/Meta-Llama-3.1-70B-Instruct-quantized.w8a8', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3d86b0f3-4df2-4ee6-9d81-cfd2d7aae326-0', usage_metadata={'input_tokens': 41, 'output_tokens': 23, 'total_tokens': 64, 'input_token_details': {}, 'output_token_details': {}})


Chat applications build upon the message history paradigm. There are three types of messages:

- System
- Human
- AI

The message history represents the conversation between the LLM and the user. The `System` message is optional. When present, it is usually used to define the _persona_ of the LLM. For example, establishing a role, defining a task, etc.

Then, the history is composed of alternating `Human` - `AI` messages.

In [3]:
# Let's build a conversation history

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples"),										# Use the system to establish the task of the LLM
    HumanMessage(content="What is Mistral and why would I need to use it"),		# This is the first "Human" message
]

# Instead of invoking with a string, pass the whole message history
response = model.invoke(messages)

# Add the response to the history
messages.append(response)

# Let's peek into the response
print(response.content)


Mistral is an open-source framework developed by the Langchain team that helps you build, deploy, and manage large language model (LLM) applications more efficiently. It's designed to streamline the development process by providing a set of tools and abstractions to work with LLMs in a more structured and organized way.

Imagine you're building a conversational AI application using a large language model like LLaMA or T5. Without Mistral, you'd need to handle tasks such as:

1. **LLM loading and caching**: Loading the model, caching its weights, and handling versioning.
2. **Input processing**: Preprocessing user input, tokenizing, and encoding it for the model.
3. **Model invocation**: Sending the input to the model, handling batching, and processing the output.
4. **Output post-processing**: Decoding, formatting, and ranking the model's output.

Mistral simplifies these tasks by providing a unified interface for working with LLMs. Here's an example of how you can use Mistral to build

Observe how the return value of the LLM model is an `AIMessage` instance. We add that response to the message history, which will be used for a follow up question

In [4]:
pprint(messages)

[SystemMessage(content='You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is Mistral and why would I need to use it', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Mistral is an open-source framework developed by the Langchain team that helps you build, deploy, and manage large language model (LLM) applications more efficiently. It\'s designed to streamline the development process by providing a set of tools and abstractions to work with LLMs in a more structured and organized way.\n\nImagine you\'re building a conversational AI application using a large language model like LLaMA or T5. Without Mistral, you\'d need to handle tasks such as:\n\n1. **LLM loading and caching**: Loading the model, caching its weights, and handling versioning.\n2. **Input processing**: Preprocessing user input, tokenizing, and encodin

Now, let's use a follow up query that relies on the coneversation history.

In [5]:
messages.append(HumanMessage("Give me a short example of a python script that calls the aforementioned LLM"))

response = model.invoke(messages)
messages.append(response)

print(response.content)

Here's a short example of a Python script that uses Mistral to call the LLaMA LLM:
```python
import mistral

# Load the LLaMA model (7B parameters)
llm = mistral.load_llm("llama-7b")

# Define a prompt to send to the model
prompt = "Write a short poem about the beauty of nature."

# Call the LLM and get the response
response = llm(prompt)

# Print the response
print(response)
```
This script will load the LLaMA model, send the prompt to the model, and print the response.

**Note**: Before running this script, you'll need to install the `langchain` library and its dependencies. You can do this by running `pip install langchain mistral` in your terminal.

Also, keep in mind that calling a large language model like LLaMA can be computationally expensive and may require significant resources (e.g., GPU, memory). You may need to adjust the model parameters or use a smaller model to suit your specific use case.

**Optional: Using the `LLMPipeline` class**

If you want to use the `LLMPipeline

Notice how we didn't explicity refered to Mistral, instead it was picked up from the message history.

Writting messages can get really tedious. We make use of message templates to simplify tasks by parameterizing the values.

We will use a template to create a document summarizer built on top of the LLM

In [6]:
# Import the chat template
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
	[("system", "Summarize the document provided by the user into a succint, at most three-sentence statement"),
	("user", "{text}")]
)

document = """
The Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;
To borrow Money on the credit of the United States;
To regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;
To establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;
To coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;
To provide for the Punishment of counterfeiting the Securities and current Coin of the United States;
To establish Post Offices and post Roads;
To promote the Progress of Science and useful Arts, by securing for limited Times to Authors and Inventors the exclusive Right to their respective Writings and Discoveries;
To constitute Tribunals inferior to the supreme Court;
To define and punish Piracies and Felonies committed on the high Seas, and Offences against the Law of Nations;
To declare War, grant Letters of Marque and Reprisal, and make Rules concerning Captures on Land and Water;
To raise and support Armies, but no Appropriation of Money to that Use shall be for a longer Term than two Years;
To provide and maintain a Navy;
To make Rules for the Government and Regulation of the land and naval Forces;
To provide for calling forth the Militia to execute the Laws of the Union, suppress Insurrections and repel Invasions;
To provide for organizing, arming, and disciplining, the Militia, and for governing such Part of them as may be employed in the Service of the United States, reserving to the States respectively, the Appointment of the Officers, and the Authority of training the Militia according to the discipline prescribed by Congress;
To exercise exclusive Legislation in all Cases whatsoever, over such District (not exceeding ten Miles square) as may, by Cession of particular States, and the Acceptance of Congress, become the Seat of the Government of the United States, and to exercise like Authority over all Places purchased by the Consent of the Legislature of the State in which the Same shall be, for the Erection of Forts, Magazines, Arsenals, dock-Yards, and other needful Buildings;—And
To make all Laws which shall be necessary and proper for carrying into Execution the foregoing Powers, and all other Powers vested by this Constitution in the Government of the United States, or in any Department or Officer thereof.
"""

prompt = prompt_template.invoke({"text": document})
response = model.invoke(prompt)

pprint(prompt)
print()
pprint(response.content)

ChatPromptValue(messages=[SystemMessage(content='Summarize the document provided by the user into a succint, at most three-sentence statement', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nThe Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;\nTo borrow Money on the credit of the United States;\nTo regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;\nTo establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;\nTo coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;\nTo provide for the Punishment of counterfeiting the Securities and current Coin of the United States;\nTo establish Post Offices an

We had to call `invoke` times: one for the prompt template and the other on the model. We can streamline this by _chaining_ them together

In [7]:
chain = prompt_template | model

response = chain.invoke({"text": document})

pprint(response.content)

('This document, Article I, Section 8 of the United States Constitution, '
 'outlines the powers granted to Congress, including taxation, commerce '
 'regulation, and national defense. Congress is also authorized to establish '
 'laws on issues such as bankruptcy, naturalization, and intellectual '
 'property, as well as to provide for the military and declare war. These '
 'powers aim to provide for the general welfare of the United States and '
 'ensure its effective governance.')


We still get as a response an `AIMessage` object. We can use _output parsers_ to transform messages into a specific format. The most basic is `StrOutputParser`.

There are other parsers to handle Json, Http responses, etc.

In [8]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

chain = prompt_template | model | str_parser

response = chain.invoke({"text": document})

# Notice how we don't have to call "content" any more
pprint(response)

('This text outlines the powers granted to Congress in the United States '
 'Constitution, including taxation, regulation of commerce, declaration of '
 'war, and establishment of laws. These powers are designed to provide for the '
 'common defense, general welfare, and execution of the government. The text '
 'also establishes the authority of Congress to make laws necessary and proper '
 'for carrying out these powers.')


The `|` operator that connects together the individual comonents is part of `LCEL`. Langchain's declarative syntax to define chains: pipelines of components that orchestrate data flow.

Let's build a sample _chain_ to generate a json object that represents an invoice from a pdf file.

In [9]:
from langchain_community.document_loaders import PyPDFLoader

# Sample PDF invoice
pdf_path = "sample-invoice.pdf"


def read_pdf(file_path:str) -> str:
	loader = PyPDFLoader(file_path)
	pages = loader.load()

	return {"text": '\n'.join([p.page_content for p in pages])}

pprint(read_pdf(pdf_path))

ValueError: File path sample-invoice.pdf is not a valid file or url

In [ ]:
# Make our function a langchain component to be compatitle with LCEL
from langchain_core.runnables import RunnableLambda

pdf_reader = RunnableLambda(read_pdf)

pprint(pdf_reader.invoke(pdf_path))

ValueError: File path sample-invoice.pdf is not a valid file or url

In [ ]:
# Create a chain to process the contents of the PDF

prompt_template = ChatPromptTemplate.from_messages(
	[("system", "You are going to read the contents of an invoice in PDF format. Return a JSON object that contains the data of the invoice using the fields and values from the text below"),
	 ("user", "{text}")]
)

# We will look at
from langchain_core.output_parsers import JsonOutputParser

chain = pdf_reader | prompt_template | model | JsonOutputParser()

invoice = chain.invoke(pdf_path)

pprint(invoice)

ValueError: File path sample-invoice.pdf is not a valid file or url

In [ ]:
# See how JSON parser create a python dictionary out of JSON data produced by the LLM
type(invoice)

NameError: name 'invoice' is not defined